# HuBMAP - Efficient Sampling Baseline (deepflash2, pytorch, fastai) [train]

> Kernel for model training with efficient region based sampling.

Requires deepflash2 (git version), zarr, and segmentation-models-pytorch


## Overview

1. Installation and package loading
2. Helper functions and patches
3. Configuration
4. Training

### Inputs
- https://www.kaggle.com/matjes/hubmap-zarr converted images (downscaled with factor 2)
- https://www.kaggle.com/matjes/hubmap-labels-pdf-0-5-0-25-0-01 masks and weights for sampling

### Versions
- V7: Fixed augmentations in deepflash2 `RandomTileDataset` config (random zoom) - LB 0.913
- V8: Adding *albumentations* transforms, switching to Cross-entropy loss

## Motivation

### Background

A glomerulus is a network of small blood vessels located at the beginning of a nephron in the kidney ([Wikipedia](https://en.wikipedia.org/wiki/Glomerulus_(kidney))
)). Glomeruli are mainly found in the renal **cortex**, while the renal **medulla** contains mainly the renal tubule. Since we are dealing with biological structures, the separation is not not absolute and the transitions are not always perfectly sharp.

![Diagram of a nephron](http://s3-us-west-2.amazonaws.com/courses-images/wp-content/uploads/sites/1842/2017/05/26234530/m9skcbftjqzrokkkopam.png)
[Diagram of a nephron from libretexts.org, Introductory and General Biology](https://bio.libretexts.org/Bookshelves/Introductory_and_General_Biology/Book%3A_General_Biology_(Boundless)/41%3A_Osmotic_Regulation_and_the_Excretory_System/41.4%3A_Human_Osmoregulatory_and_Excretory_Systems/41.4B%3A_Nephron%3A_The_Functional_Unit_of_the_Kidney)

### Key Idea

A common approach to deal with the very large (>500MB - 5GB) TIFF files in the dataset is to decompose the images in smaller patches/tiles, for instance by using a sliding window apporach.
> **Knowing that the glomeruli are mainly found in the cortex, we should focus on this region during training**. 

Instead of preprocessing the images and saving them into fixed tiles, we sample tiles from the entire images with a higher probability on tiles that contain glumeroli and cortex. Have a look at [this kernel](https://www.kaggle.com/matjes/hubmap-labels-pdf-0-5-0-25-0-01) for more details.


## Advantages of this approach

In combination with [deepflash2](https://github.com/matjesg/deepflash2/tree/master/) and the deepflash2 [pytorch datasets](https://matjesg.github.io/deepflash2/data.html#Datasets) in particular, this approach has several advantages:
- no preprocessing of the data (only saving them to .zarr files for memory efficient loading)
    - flexible tile shapes (input shapes, e.g. 1024, 512, 256) at runtime
    - flexible scaling (e.g., by facors of 2,3,4)
- faster convergence during traing (~30 min for training a competitive model)
    - focusing on the relevant regions (e.g., tiles that contain glumeroli and cortex)
    - "additional" data augmentation from random sampling (compared to fixed windows)

### Installation and package loading

In [1]:
!pip install -q git+https://github.com/matjesg/deepflash2.git
!pip install segmentation_models_pytorch

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\soodn\\Anaconda3\\envs\\tf-gpu\\Lib\\site-packages\\cv2\\cv2.cp38-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



In [1]:
# imports
import zarr, cv2
import numpy as np, pandas as pd, segmentation_models_pytorch as smp
from fastai.vision.all import *
from deepflash2.all import *
import albumentations as alb

## Helper functions and patches

![](http://)Patches for deepflash2 classes, see https://fastcore.fast.ai/basics.html#patch

In [2]:
@patch
def read_img(self:BaseDataset, file, *args, **kwargs):
    return zarr.open(str(file), mode='r')

@patch
def _name_fn(self:BaseDataset, g):
    "Name of preprocessed and compressed data."
    return f'{g}'

@patch
def apply(self:DeformationField, data, offset=(0, 0), pad=(0, 0), order=1):
    "Apply deformation field to image using interpolation"
    outshape = tuple(int(s - p) for (s, p) in zip(self.shape, pad))
    coords = [np.squeeze(d).astype('float32').reshape(*outshape) for d in self.get(offset, pad)]
    # Get slices to avoid loading all data (.zarr files)
    sl = []
    for i in range(len(coords)):
        cmin, cmax = int(coords[i].min()), int(coords[i].max())
        dmax = data.shape[i]
        if cmin<0: 
            cmax = max(-cmin, cmax)
            cmin = 0 
        elif cmax>dmax:
            cmin = min(cmin, 2*dmax-cmax)
            cmax = dmax
            coords[i] -= cmin
        else: coords[i] -= cmin
        sl.append(slice(cmin, cmax))    
    if len(data.shape) == len(self.shape) + 1:
        tile = np.empty((*outshape, data.shape[-1]))
        for c in range(data.shape[-1]):
            # Adding divide
            tile[..., c] = cv2.remap(data[sl[0],sl[1], c]/255, coords[1],coords[0], interpolation=order, borderMode=cv2.BORDER_REFLECT)
    else:
        tile = cv2.remap(data[sl[0], sl[1]], coords[1], coords[0], interpolation=order, borderMode=cv2.BORDER_REFLECT)
    return tile

### Config

In [3]:
class CONFIG():
    
    # data paths
    data_path = Path(r'C:\Users\soodn\Downloads\Naveksha\Kaggle HuBMAP\Data\hubmap-kidney-segmentation-data')
    data_path_zarr = Path('images_scale2')
    mask_preproc_dir = 'masks_scale2'
    
    # deepflash2 dataset
    scale = 1.5 # data is already downscaled to 2, so absulute downscale is 3
    tile_shape = (512, 512)
    padding = (0,0) # Border overlap for prediction
    n_jobs = 1
    sample_mult = 100 # Sample 100 tiles from each image, per epoch
    val_length = 500 # Randomly sample 500 validation tiles
    stats = np.array([0.61561477, 0.5179343 , 0.64067212]), np.array([0.2915353 , 0.31549066, 0.28647661])
    
    # deepflash2 augmentation options
    zoom_sigma = 0.1
    flip = True
    max_rotation = 360
    deformation_grid_size = (150,150)
    deformation_magnitude = (10,10)
    

    # pytorch model (segmentation_models_pytorch)
    encoder_name = "efficientnet-b4"
    encoder_weights = 'imagenet'
    in_channels = 3
    classes = 2
    
    # fastai Learner 
    mixed_precision_training = True
    batch_size = 16
    weight_decay = 0.01
    loss_func = CrossEntropyLossFlat(axis=1)
    metrics = [Iou(), Dice_f1()]
    optimizer = ranger
    max_learning_rate = 1e-3
    epochs = 12
    
cfg = CONFIG()

In [4]:
# Albumentations augmentations
# Inspired by https://www.kaggle.com/iafoss/hubmap-pytorch-fast-ai-starter
# deepflash2 augmentations are only affine transformations
tfms = alb.OneOf([
    alb.HueSaturationValue(10,15,10),
    alb.CLAHE(clip_limit=2),
    alb.RandomBrightnessContrast(),            
    ], p=0.3)

In [5]:
df_train = pd.read_csv(cfg.data_path/'train.csv')
df_info = pd.read_csv(cfg.data_path/'HuBMAP-20-dataset_information.csv')

files = [x for x in cfg.data_path_zarr.iterdir() if x.is_dir() if not x.name.startswith('.')]
label_fn = lambda o: o

In [6]:
files

[Path('images_scale2/0486052bb'),
 Path('images_scale2/095bf7a1f'),
 Path('images_scale2/1e2425f28'),
 Path('images_scale2/26dc41664'),
 Path('images_scale2/2ec3f1bb9'),
 Path('images_scale2/2f6ecfcdf'),
 Path('images_scale2/3589adb90'),
 Path('images_scale2/4ef6695ce'),
 Path('images_scale2/54f2eec69'),
 Path('images_scale2/57512b7f1'),
 Path('images_scale2/8242609fa'),
 Path('images_scale2/aa05346ff'),
 Path('images_scale2/aaa6a05cc'),
 Path('images_scale2/afa5e8098'),
 Path('images_scale2/b2dc8411c'),
 Path('images_scale2/b9a3865fc'),
 Path('images_scale2/c68fe75ea'),
 Path('images_scale2/cb2d976f4'),
 Path('images_scale2/d488c759a'),
 Path('images_scale2/e79de561c')]

## Training

In [7]:
# Model
model = smp.Unet(encoder_name=cfg.encoder_name, 
                 encoder_weights=cfg.encoder_weights, 
                 in_channels=cfg.in_channels, 
                 classes=cfg.classes)

In [8]:
# Datasets
ds_kwargs = {
    'tile_shape':cfg.tile_shape,
    'padding':cfg.padding,
    'scale': cfg.scale,
    'n_jobs': cfg.n_jobs, 
    'preproc_dir': cfg.mask_preproc_dir, 
    'val_length':cfg.val_length, 
    'sample_mult':cfg.sample_mult,
    'loss_weights':False,
    'zoom_sigma': cfg.zoom_sigma,
    'flip' : cfg.flip,
    'max_rotation': cfg.max_rotation,
    'deformation_grid_size' : cfg.deformation_grid_size,
    'deformation_magnitude' : cfg.deformation_magnitude,
    'albumentations_tfms': tfms
}
train_ds = RandomTileDataset(files, label_fn=label_fn, **ds_kwargs)
valid_ds = TileDataset(files, label_fn=label_fn, **ds_kwargs, is_zarr=True)

Using preprocessed masks from masks_scale2
Preprocessing 2ec3f1bb9


ValueError: Could not find a format to read the specified file in single-image mode

In [ ]:
# Dataloader and learner
dls = DataLoaders.from_dsets(train_ds, valid_ds, bs=cfg.batch_size, after_batch=Normalize.from_stats(*cfg.stats))
if torch.cuda.is_available(): dls.cuda(), model.cuda()
cbs = [SaveModelCallback(monitor='iou'), ElasticDeformCallback]
learn = Learner(dls, model, metrics=cfg.metrics, wd=cfg.weight_decay, loss_func=cfg.loss_func, opt_func=ranger, cbs=cbs)
if cfg.mixed_precision_training: learn.to_fp16()
  

In [ ]:
# Fit
learn.fit_one_cycle(cfg.epochs, lr_max=cfg.max_learning_rate)
learn.recorder.plot_metrics()

In [ ]:
# Save Model
state = {'model': learn.model.state_dict(), 'stats':cfg.stats}
torch.save(state, f'unet_{cfg.encoder_name}.pth', pickle_protocol=2, _use_new_zipfile_serialization=False)